# Sources
https://www.kaggle.com/rounakbanik/movie-recommender-systems

## Simple Recommender

The Simple Recommender offers generalized recommnendations to every user based on movie popularity and (sometimes) genre. The basic idea behind this recommender is that movies that are more popular and more critically acclaimed will have a higher probability of being liked by the average audience. This model does not give personalized recommendations based on the user. 

The implementation of this model is extremely trivial. All we have to do is sort our movies based on ratings and popularity and display the top movies of our list. As an added step, we can pass in a genre argument to get the top movies of a particular genre. 

In [187]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

import warnings; warnings.simplefilter('ignore')

In [188]:
md = pd. read_csv('../data/movies_metadata.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [189]:
md.columns

Index([u'adult', u'belongs_to_collection', u'budget', u'genres', u'homepage',
       u'id', u'imdb_id', u'original_language', u'original_title', u'overview',
       u'popularity', u'poster_path', u'production_companies',
       u'production_countries', u'release_date', u'revenue', u'runtime',
       u'spoken_languages', u'status', u'tagline', u'title', u'video',
       u'vote_average', u'vote_count'],
      dtype='object')

In [190]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

I use the TMDB Ratings to come up with our **Top Movies Chart.** I will use IMDB's *weighted rating* formula to construct my chart. Mathematically, it is represented as follows:

Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

where,
* *v* is the number of votes for the movie
* *m* is the minimum votes required to be listed in the chart
* *R* is the average rating of the movie
* *C* is the mean vote across the whole report

The next step is to determine an appropriate value for *m*, the minimum votes required to be listed in the chart. We will use **95th percentile** as our cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 95% of the movies in the list.

I will build our overall Top 250 Chart and will define a function to build charts for a particular genre. Let's begin!

In [191]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [192]:
m = vote_counts.quantile(0.95)
m

434.0

In [193]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [194]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres','homepage','poster_path']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 8)

In [195]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [196]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [197]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [198]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,homepage,poster_path,wr
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",http://inceptionmovie.warnerbros.com/,/qmDpIHrmpJINaRKAfWQfftjCdyi.jpg,7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",http://thedarkknight.warnerbros.com/dvdsite/,/1hRoyzDtpgMU7Dz4JF22RANzQO7.jpg,7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",http://www.interstellarmovie.net/,/nBNZadXqJSdt05SHLqgT0HuC5Gm.jpg,7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],http://www.foxmovies.com/movies/fight-club,/adw6Lq9FiC9zjYEpOqfq03ituwp.jpg,7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",http://www.lordoftherings.net/,/bxVxZb5O9OxCO0oRUNdCnpy9NST.jpg,7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",NaN,/dM2w364MScsjFf8pfMbaWUcWrR.jpg,7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",NaN,/9O7gLzmreU0nGkIB6K3BsJbzvNv.jpg,7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",http://www.lordoftherings.net,/uexxR7Kw1qYbZk0RYaF9Rx5ykbj.jpg,7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",NaN,/yE5d3BUhE8hCnkMUJOo1QDoOGNz.jpg,7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",http://www.lordoftherings.net/,/wf3v0Pn09jnT5HSaYal7Ami3bdA.jpg,7.851924


In [199]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [200]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity','homepage','poster_path']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [201]:
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,homepage,poster_path,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,NaN,/2gvbZMtV1Zsl7FedJa5ysbpBx2G.jpg,8.565285
351,Forrest Gump,1994,8147,8,48.3072,NaN,/yE5d3BUhE8hCnkMUJOo1QDoOGNz.jpg,7.971357
876,Vertigo,1958,1162,8,18.2082,NaN,/obhM86qyv8RsE69XSMTtT9FdE0b.jpg,7.811667
40251,Your Name.,2016,1030,8,34.461252,https://www.funimationfilms.com/movie/yourname/,/xq1Ugd62d23K2knRUx6xxuALTZB.jpg,7.789489
883,Some Like It Hot,1959,835,8,11.8451,NaN,/pxc9EFCMYkItESpqqrI783yl8Gh.jpg,7.745154
1132,Cinema Paradiso,1988,834,8,14.177,NaN,/xKAweeF2ZPMNn6ce4GclSbr59Pv.jpg,7.744878
19901,Paperman,2012,734,8,7.19863,http://www.disneyanimation.com/projects/shorts...,/3TpMBcAYH4cxCw5WoRacWodMTCG.jpg,7.713951
37863,Sing Street,2016,669,8,10.672862,NaN,/s0C78plmx3dFcO3WMnoXCz56FiN.jpg,7.689483
882,The Apartment,1960,498,8,11.9943,NaN,/6OH3HlplEGoGjqeqRmWaVBGqbav.jpg,7.599317
38718,The Handmaiden,2016,453,8,16.727405,NaN,/wvzfK5QR6dGLwND8MCzWjsQWG4Q.jpg,7.566166


In [219]:
genreSet = list(set(s))
dfGenre = build_chart(genreSet[2]).head(5)
dfGenre['Genre'] = genreSet[2]
for i in range(3,len(genreSet)):
    try:
        x = build_chart(genreSet[i]).head(5)
        x['Genre'] = genreSet[i]
        dfGenre = dfGenre.append(x)
    except:
        continue

In [220]:
dfGenre

,title,year,vote_count,vote_average,popularity,homepage,poster_path,wr,Genre
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,NaN,/2gvbZMtV1Zsl7FedJa5ysbpBx2G.jpg,8.565285,Romance
351,Forrest Gump,1994,8147,8,48.3072,NaN,/yE5d3BUhE8hCnkMUJOo1QDoOGNz.jpg,7.971357,Romance
876,Vertigo,1958,1162,8,18.2082,NaN,/obhM86qyv8RsE69XSMTtT9FdE0b.jpg,7.811667,Romance
40251,Your Name.,2016,1030,8,34.461252,https://www.funimationfilms.com/movie/yourname/,/xq1Ugd62d23K2knRUx6xxuALTZB.jpg,7.789489,Romance
883,Some Like It Hot,1959,835,8,11.8451,NaN,/pxc9EFCMYkItESpqqrI783yl8Gh.jpg,7.745154,Romance
359,The Lion King,1994,5520,8,21.6058,http://movies.disney.com/the-lion-king,/bKPtXn9n4M4s8vvZrbw40mYsefB.jpg,7.909339,Animation
5481,Spirited Away,2001,3968,8,41.0489,http://movies.disney.com/spirited-away,/ynXoOxmDHNQ4UAy0oU6avW71HVW.jpg,7.875933,Animation
9698,Howl's Moving Castle,2004,2049,8,16.136,NaN,/iMarB2ior30OAXjPa7QIdeyUfM1.jpg,7.772103,Animation
2884,Princess Mononoke,1997,2041,8,17.1667,NaN,/gzlJkVfWV5VEG5xK25cvFGJgkDz.jpg,7.771305,Animation
5833,My Neighbor Totoro,1988,1730,8,13.5073,NaN,/2i0OOjvi7CqNQA6ZtYJtL65P9oZ.jpg,7.735274,Animation


In [222]:
dfGenre.homepage.fillna('https://www.google.com/search?q=' + dfGenre.title.str.replace(' ','+'), inplace=True)

In [223]:
dfGenre

,title,year,vote_count,vote_average,popularity,homepage,poster_path,wr,Genre
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,https://www.google.com/search?q=Dilwale+Dulhan...,/2gvbZMtV1Zsl7FedJa5ysbpBx2G.jpg,8.565285,Romance
351,Forrest Gump,1994,8147,8,48.3072,https://www.google.com/search?q=Forrest+Gump,/yE5d3BUhE8hCnkMUJOo1QDoOGNz.jpg,7.971357,Romance
876,Vertigo,1958,1162,8,18.2082,https://www.google.com/search?q=Vertigo,/obhM86qyv8RsE69XSMTtT9FdE0b.jpg,7.811667,Romance
40251,Your Name.,2016,1030,8,34.461252,https://www.funimationfilms.com/movie/yourname/,/xq1Ugd62d23K2knRUx6xxuALTZB.jpg,7.789489,Romance
883,Some Like It Hot,1959,835,8,11.8451,https://www.google.com/search?q=Some+Like+It+Hot,/pxc9EFCMYkItESpqqrI783yl8Gh.jpg,7.745154,Romance
359,The Lion King,1994,5520,8,21.6058,http://movies.disney.com/the-lion-king,/bKPtXn9n4M4s8vvZrbw40mYsefB.jpg,7.909339,Animation
5481,Spirited Away,2001,3968,8,41.0489,http://movies.disney.com/spirited-away,/ynXoOxmDHNQ4UAy0oU6avW71HVW.jpg,7.875933,Animation
9698,Howl's Moving Castle,2004,2049,8,16.136,https://www.google.com/search?q=Howl's+Moving+...,/iMarB2ior30OAXjPa7QIdeyUfM1.jpg,7.772103,Animation
2884,Princess Mononoke,1997,2041,8,17.1667,https://www.google.com/search?q=Princess+Mononoke,/gzlJkVfWV5VEG5xK25cvFGJgkDz.jpg,7.771305,Animation
5833,My Neighbor Totoro,1988,1730,8,13.5073,https://www.google.com/search?q=My+Neighbor+To...,/2i0OOjvi7CqNQA6ZtYJtL65P9oZ.jpg,7.735274,Animation


In [225]:
dfGenre.to_csv("../data/top5Genre.csv")

## Content Based Recommender

The recommender we built in the previous section suffers some severe limitations. For one, it gives the same recommendation to everyone, regardless of the user's personal taste. If a person who loves romantic movies (and hates action) were to look at our Top 15 Chart, s/he wouldn't probably like most of the movies. If s/he were to go one step further and look at our charts by genre, s/he wouldn't still be getting the best recommendations.

For instance, consider a person who loves *Dilwale Dulhania Le Jayenge*, *My Name is Khan* and *Kabhi Khushi Kabhi Gham*. One inference we can obtain is that the person loves the actor Shahrukh Khan and the director Karan Johar. Even if s/he were to access the romance chart, s/he wouldn't find these as the top recommendations.

To personalise our recommendations more, I am going to build an engine that computes similarity between movies based on certain metrics and suggests movies that are most similar to a particular movie that a user liked. Since we will be using movie metadata (or content) to build this engine, this also known as **Content Based Filtering.**

I will build two Content Based Recommenders based on:
* Movie Overviews and Taglines
* Movie Cast, Crew, Keywords and Genre

Also, as mentioned in the introduction, I will be using a subset of all the movies available to us due to limiting computing power available to me. 

In [227]:
links_small = pd.read_csv('../data/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [228]:
md = md.drop([19730, 29503, 35587])

In [229]:
#Check EDA Notebook for how and why I got these indices.
md['id'] = md['id'].astype('int')

In [230]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

We have **9099** movies avaiable in our small movies metadata dataset which is 5 times smaller than our original dataset of 45000 movies.

### Movie Description Based Recommender

Let us first try to build a recommender using movie descriptions and taglines. We do not have a quantitative metric to judge our machine's performance so this will have to be done qualitatively.

In [232]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [233]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [234]:
tfidf_matrix.shape

(9099, 268124)

#### Cosine Similarity

I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two movies. Mathematically, it is defined as follows:

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $

Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. Therefore, we will use sklearn's **linear_kernel** instead of cosine_similarities since it is much faster.

In [236]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [237]:
cosine_sim[0]

array([ 1.        ,  0.00680476,  0.        , ...,  0.        ,
        0.00344913,  0.        ])

We now have a pairwise cosine similarity matrix for all the movies in our dataset. The next step is to write a function that returns the 30 most similar movies based on the cosine similarity score.

In [239]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [240]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

We're all set. Let us now try and get the top recommendations for a few movies and see how good the recommendations are.

In [242]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

We see that for **The Dark Knight**, our system is able to identify it as a Batman film and subsequently recommend other Batman films as its top recommendations. But unfortunately, that is all this system can do at the moment. This is not of much use to most people as it doesn't take into considerations very important features such as cast, crew, director and genre, which determine the rating and the popularity of a movie. Someone who liked **The Dark Knight** probably likes it more because of Nolan and would hate **Batman Forever** and every other substandard movie in the Batman Franchise.

Therefore, we are going to use much more suggestive metadata than **Overview** and **Tagline**. In the next subsection, we will build a more sophisticated recommender that takes **genre**, **keywords**, **cast** and **crew** into consideration.

### Metadata Based Recommender

To build our standard metadata based content recommender, we will need to merge our current dataset with the crew and the keyword datasets. Let us prepare this data as our first step.

In [243]:
credits = pd.read_csv('../data/credits.csv')
keywords = pd.read_csv('../data/keywords.csv')

In [244]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [245]:
md.shape

(45463, 25)

In [246]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [247]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

We now have our cast, crew, genres and credits, all in one dataframe. Let us wrangle this a little more using the following intuitions:

1. **Crew:** From the crew, we will only pick the director as our feature since the others don't contribute that much to the *feel* of the movie.
2. **Cast:** Choosing Cast is a little more tricky. Lesser known actors and minor roles do not really affect people's opinion of a movie. Therefore, we must only select the major characters and their respective actors. Arbitrarily we will choose the top 3 actors that appear in the credits list. 

In [248]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [249]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [250]:
smd['director'] = smd['crew'].apply(get_director)

In [251]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [252]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

My approach to building the recommender is going to be extremely *hacky*. What I plan on doing is creating a metadata dump for every movie which consists of **genres, director, main actors and keywords.** I then use a **Count Vectorizer** to create our count matrix as we did in the Description Recommender. The remaining steps are similar to what we did earlier: we calculate the cosine similarities and return movies that are most similar.

These are steps I follow in the preparation of my genres and credits data:
1. **Strip Spaces and Convert to Lowercase** from all our features. This way, our engine will not confuse between **Johnny Depp** and **Johnny Galecki.** 
2. **Mention Director 3 times** to give it more weight relative to the entire cast.

In [253]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [254]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

#### Keywords

We will do a small amount of pre-processing of our keywords before putting them to any use. As a first step, we calculate the frequenct counts of every keyword that appears in the dataset.

In [255]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [256]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

Keywords occur in frequencies ranging from 1 to 610. We do not have any use for keywords that occur only once. Therefore, these can be safely removed. Finally, we will convert every word to its stem so that words such as *Dogs* and *Dog* are considered the same.

In [257]:
s = s[s > 1]

In [258]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

u'dog'

In [259]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [267]:
import sys
reload(sys)
sys.setdefaultencoding("ISO-8859-1")
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

TypeError: descriptor 'lower' requires a 'str' object but received a 'unicode'

In [268]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [269]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [270]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [271]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [272]:
get_recommendations('The Dark Knight').head(10)

6218                         Batman Begins
8031                 The Dark Knight Rises
1134                        Batman Returns
6623                          The Prestige
7659            Batman: Under the Red Hood
2085                             Following
1260                        Batman & Robin
4145                              Insomnia
7648                             Inception
9024    Batman v Superman: Dawn of Justice
Name: title, dtype: object

In [ ]:
idx = indices[title]
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:31]
movie_indices = [i[0] for i in sim_scores]
#return titles.iloc[movie_indices]

In [289]:
titleDF = pd.DataFrame(titles)


In [291]:
cosine_simDF = pd.DataFrame(cosine_sim)

In [293]:
cosine_simDF

,0,1,2,3,4,5,6,7,8,9,...,9209,9210,9211,9212,9213,9214,9215,9216,9217,9218
0,1.000000,0.037980,0.045326,0.019854,0.018990,0.000000,0.020332,0.025055,0.000000,0.000000,...,0.000000,0.000000,0.018990,0.000000,0.016669,0.026038,0.000000,0.000000,0.000000,0.000000
1,0.037980,1.000000,0.000000,0.022244,0.000000,0.000000,0.000000,0.056143,0.023980,0.016011,...,0.000000,0.000000,0.085106,0.000000,0.000000,0.000000,0.000000,0.027086,0.044488,0.000000
2,0.045326,0.000000,1.000000,0.053093,0.025392,0.000000,0.054373,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.022288,0.000000,0.033501,0.032325,0.000000,0.000000
3,0.019854,0.022244,0.053093,1.000000,0.044488,0.015813,0.071449,0.029348,0.000000,0.000000,...,0.000000,0.017379,0.044488,0.030500,0.019525,0.030500,0.029348,0.056637,0.023256,0.000000
4,0.018990,0.000000,0.025392,0.044488,1.000000,0.000000,0.045561,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.018676,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.015813,0.000000,1.000000,0.000000,0.039912,0.034095,0.022764,...,0.000000,0.106354,0.045376,0.062217,0.000000,0.020739,0.019956,0.019256,0.031627,0.000000
6,0.020332,0.000000,0.054373,0.071449,0.045561,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.035595,0.000000,0.000000,0.019996,0.000000,0.030056,0.029001,0.000000,0.000000
7,0.025055,0.056143,0.000000,0.029348,0.000000,0.039912,0.000000,1.000000,0.094916,0.042248,...,0.000000,0.021932,0.112287,0.038490,0.000000,0.038490,0.000000,0.107211,0.146742,0.000000
8,0.000000,0.023980,0.000000,0.000000,0.000000,0.034095,0.000000,0.094916,1.000000,0.054135,...,0.000000,0.018735,0.047960,0.032880,0.000000,0.000000,0.031639,0.030528,0.075212,0.000000
9,0.000000,0.016011,0.000000,0.000000,0.000000,0.022764,0.000000,0.042248,0.054135,1.000000,...,0.000000,0.037526,0.032022,0.021953,0.000000,0.000000,0.021124,0.020383,0.033478,0.000000


In [296]:
totalDF = pd.concat([titleDF, cosine_simDF],axis =1)

In [297]:
totalDF

,title,0,1,2,3,4,5,6,7,8,...,9209,9210,9211,9212,9213,9214,9215,9216,9217,9218
0,Toy Story,1.000000,0.037980,0.045326,0.019854,0.018990,0.000000,0.020332,0.025055,0.000000,...,0.000000,0.000000,0.018990,0.000000,0.016669,0.026038,0.000000,0.000000,0.000000,0.000000
1,Jumanji,0.037980,1.000000,0.000000,0.022244,0.000000,0.000000,0.000000,0.056143,0.023980,...,0.000000,0.000000,0.085106,0.000000,0.000000,0.000000,0.000000,0.027086,0.044488,0.000000
2,Grumpier Old Men,0.045326,0.000000,1.000000,0.053093,0.025392,0.000000,0.054373,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.022288,0.000000,0.033501,0.032325,0.000000,0.000000
3,Waiting to Exhale,0.019854,0.022244,0.053093,1.000000,0.044488,0.015813,0.071449,0.029348,0.000000,...,0.000000,0.017379,0.044488,0.030500,0.019525,0.030500,0.029348,0.056637,0.023256,0.000000
4,Father of the Bride Part II,0.018990,0.000000,0.025392,0.044488,1.000000,0.000000,0.045561,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.018676,0.000000,0.000000,0.000000,0.000000,0.000000
5,Heat,0.000000,0.000000,0.000000,0.015813,0.000000,1.000000,0.000000,0.039912,0.034095,...,0.000000,0.106354,0.045376,0.062217,0.000000,0.020739,0.019956,0.019256,0.031627,0.000000
6,Sabrina,0.020332,0.000000,0.054373,0.071449,0.045561,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.035595,0.000000,0.000000,0.019996,0.000000,0.030056,0.029001,0.000000,0.000000
7,Tom and Huck,0.025055,0.056143,0.000000,0.029348,0.000000,0.039912,0.000000,1.000000,0.094916,...,0.000000,0.021932,0.112287,0.038490,0.000000,0.038490,0.000000,0.107211,0.146742,0.000000
8,Sudden Death,0.000000,0.023980,0.000000,0.000000,0.000000,0.034095,0.000000,0.094916,1.000000,...,0.000000,0.018735,0.047960,0.032880,0.000000,0.000000,0.031639,0.030528,0.075212,0.000000
9,GoldenEye,0.000000,0.016011,0.000000,0.000000,0.000000,0.022764,0.000000,0.042248,0.054135,...,0.000000,0.037526,0.032022,0.021953,0.000000,0.000000,0.021124,0.020383,0.033478,0.000000


In [298]:
totalDF.to_csv('../data/contentRec.csv')